# 🚀Analise da tabela Stage Area

### 📌 Passo 1: Configurar a Conexão com o Banco

In [2]:
!pip install sqlalchemy
import pandas as pd
from sqlalchemy import create_engine
import os

# 🔹 Configurar conexão com o banco
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

# 🔹 Carregar dados da Staging Area
df_stg = pd.read_sql("SELECT * FROM stg_acidentes", con=engine)

# 🔍 Exibir as primeiras linhas para análise
df_stg.head()


,id,concessionaria,data,horario,n_da_ocorrencia,tipo_de_ocorrencia,km,trecho,sentido,tipo_de_acidente,...,outros,tracao_animal,transporte_de_cargas_especiais,trator_maquinas,utilitarios,ilesos,levemente_feridos,moderadamente_feridos,gravemente_feridos,mortos
0,256630c4-69b2-4d09-9481-4659040fb051,AUTOPISTA_FERNAO_DIAS,01/01/2010,03:02:00,20,sem vítima,506.5,BR-381/MG,Sul,"Choque - Defensa, barreira ou submarino""""",...,0,0,0,0,0,1,0,0,0,0
1,d5924260-52cf-4a95-981e-f97e652710dc,AUTOPISTA_FERNAO_DIAS,01/01/2010,06:16:00,39,sem vítima,767,BR-381/MG,Sul,Saida de Pista,...,0,0,0,0,0,2,0,0,0,0
2,955e1026-8088-4555-896c-47366a3b62df,AUTOPISTA_FERNAO_DIAS,01/01/2010,06:37:00,42,com vítima,567,BR-381/MG,Sul,Choque - Suporte de Sinalização,...,0,0,0,0,0,0,1,0,0,0
3,1959f647-2d87-4181-9ac9-1032b3731ea7,AUTOPISTA_FERNAO_DIAS,01/01/2010,06:49:00,44,sem vítima,492,BR-381/MG,Norte,Saida de Pista,...,0,0,0,0,0,1,0,0,0,0
4,e02e1b60-dcb8-41ae-8814-9c4a1bbb2437,AUTOPISTA_FERNAO_DIAS,01/01/2010,07:11:00,48,sem vítima,76.5,BR-381/SP,Sul,"Choque - Defensa, barreira ou submarino""""",...,0,0,0,0,0,1,0,0,0,0
